In [180]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy.optimize import rosen
import torch
from botorch.acquisition import UpperConfidenceBound
from botorch.fit import fit_gpytorch_model
from botorch.models import SingleTaskGP
from botorch.optim import optimize_acqf
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood

In [181]:
def calculate_target_function(train_X, dtype=torch.float32, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    target_fn = torch.from_numpy(rosen(train_X)[..., None])
    target_fn = target_fn.to(dtype=dtype, device=device)
    return target_fn

In [182]:
def BO_procedure(train_X, train_Y, bounds):
    gp = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_model(mll)

    UCB = UpperConfidenceBound(gp, beta=5, dtype=torch.float32)  # higher beta means more exploration (example: 10000)

    candidate, acq_value = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20, dtype=torch.float32
    )

    candidate_y = calculate_target_function(candidate)
    # print(f'    Candidate: ({candidate[0][0].numpy()}, {candidate_y[0][0].numpy()})')

    new_X = torch.cat([train_X, candidate])
    new_Y = torch.cat([train_Y, candidate_y])

    return gp, new_X, new_Y

In [183]:
def plot(model, train_X, train_Y, bounds):
    from matplotlib import pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    %matplotlib inline

    target_x = torch.linspace(bounds[0][0], bounds[1][0], 101)
    target_y = calculate_target_function(target_x, target_x)

    # Initialize plot
    ax = plt.subplot(111, projection='3d')
    # test model on 101 regular spaced points on the interval [0, 1]
    test_X = torch.linspace(bounds[0][0], bounds[1][0], 101)

    

    # no need for gradients
    with torch.no_grad():
        # plot target function (inclusion of noise decreases accuracy)
        ax.plot(target_x.cpu().numpy(), target_y.cpu().numpy(), 'r')

        # compute posterior
        posterior = model.posterior(test_X)
        # Get upper and lower confidence bounds (2 standard deviations from the mean)
        lower, upper = posterior.mvn.confidence_region()
        # Plot training points as black stars
        ax.plot(train_X.cpu().numpy(), train_Y.cpu().numpy(), 'k*')
        # Plot posterior means as blue line
        ax.plot(test_X.cpu().numpy(), posterior.mean.cpu().numpy(), 'b')
        # Shade between the lower and upper confidence bounds
        ax.fill_between(test_X.cpu().numpy(), lower.cpu().numpy(), upper.cpu().numpy(), alpha=0.5)

    ax.legend(['Target Function', 'Observed Data', 'Mean', 'Confidence'])
    plt.tight_layout()

In [184]:
# use a GPU if available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = dtype=torch.float32

NUM_ITERATIONS = 20
BOUNDS = torch.tensor([[-1.0, -1.0], [1.0, 1.0]])
x = torch.linspace(BOUNDS[0][0], BOUNDS[1][0], 2, dtype=DTYPE, device=DEVICE).unsqueeze(1)
y = torch.linspace(BOUNDS[0][1], BOUNDS[1][1], 2, dtype=DTYPE, device=DEVICE).unsqueeze(1)
train_X = torch.cat((x, y), 1)  # 0 for concatenate as row, 1 as column
train_Y = calculate_target_function(train_X)
print(train_X)
print(train_Y)
for i in range(NUM_ITERATIONS):
    print(f'Executing Iteration {i + 1}:')
    model, train_X, train_Y = BO_procedure(train_X, train_Y, BOUNDS)

# plot(model, train_X, train_Y, BOUNDS)

tensor([[-1., -1.],
        [ 1.,  1.]])
tensor([[4.],
        [4.]])
Executing Iteration 1:
Executing Iteration 2:


RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0